In [1]:
import sys
sys.path.append('../Util')
import pickle
import os

In [2]:
from IPython.display import Image

In [3]:
from evaluation import correlation, lr, perceptron, get_anova_dims, get_mi_dims
from preparation import prepare_dataset, read_datasets

In [4]:
from we import get_we, initiate_model

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
from scipy.stats import f_oneway

In [7]:
from sklearn.metrics import mean_absolute_error, accuracy_score
from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer

In [8]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
models = [
    {
        'name': 'flaubert/flaubert_small_cased',
        'label': 'flau_small_c'
    },
    {
    
        'name': 'flaubert/flaubert_base_uncased', 
        'label': 'flau_base_u'

    },
    {
        'name': 'flaubert/flaubert_base_cased',
        'label': 'flau_base_c'
    },
    {
        'name': 'flaubert/flaubert_large_cased',
        'label': 'flau_large_c'
    },
    {
        'name': 'camembert/camembert-base',
        'label': 'cam_base'
    },
    {
        'name': 'xlm-roberta-large',
        'label': 'xlm_large'
    },
    {
    
        'name': 'xlm-roberta-base', 
        'label': 'xlm_base'

    },
    {
        'name': 'bert-base-multilingual-uncased',
        'label': 'bert_base_u'
    },
    {
        'name': 'distilbert-base-multilingual-cased',
        'label': 'distilbert_base'
    },
    {
        'name': 'bert-base-multilingual-cased',
        'label': 'bert_base_c'
    }
    
]

In [10]:
labels = [m['label'] for m in models]

In [11]:
we_with_features = read_datasets(
                            path = '../Data',
                            model_labels = labels,
                            file_name = 'all_unique_pos_we.csv'
                    )

In [12]:
we_with_features[-1].head()

,0,1,2,3,4,5,6,7,8,9,...,764,765,766,767,Number,Gender,Lemma,POS,Tense,Person
Word,,,,,,,,,,,,,,,,,,,,,
2D,0.909603,0.551025,0.436937,0.789782,0.311517,0.417476,0.629341,0.913722,0.349834,0.838988,...,0.285154,0.057887,0.030874,0.310204,invariable,feminine,2D,NOUN,NaN,NaN
3D,0.901779,0.549880,0.427826,0.797102,0.313900,0.419226,0.623478,0.916539,0.355807,0.844714,...,0.287610,0.054251,0.018604,0.305226,invariable,feminine,3D,NOUN,NaN,NaN
aa,0.899570,0.555891,0.418186,0.805754,0.319232,0.412976,0.618457,0.914183,0.367882,0.845985,...,0.287883,0.057476,0.013614,0.290463,invariable,masculine,aa,NOUN,NaN,NaN
abandon,0.898879,0.556306,0.417821,0.805256,0.318463,0.412505,0.618276,0.914983,0.367654,0.846577,...,0.288183,0.057827,0.014050,0.290994,singular,masculine,abandon,NOUN,NaN,NaN
abbaye,0.895888,0.554461,0.420659,0.799339,0.322441,0.418141,0.618820,0.921033,0.367005,0.845904,...,0.293985,0.057312,0.012175,0.298473,singular,feminine,abbaye,NOUN,NaN,NaN


In [13]:
# There are 6 extra feautres in addition to embedding dimensions in the file
feature_col_count = 6

# Feature to investigate in this notebook
feature = 'Number'

In this notebook we will calculate **InfEnc** metric for word embeddings of different models, in order to compare the quality of information encoding about `Number of Nouns and Adjectives`.

For this we will pick concatenate dimensions of word embedding of nouns and adjectives. Note that words that are both adjectives and nouns were discarded previously.

# Nouns

In [14]:
pos = ['NOUN', 'ADJ']

Split each model into train and test using k_fold cross validation:

In [15]:
X_na_train = []
y_na_train = []

X_na_test = []
y_na_test = []

In [17]:
n_folds = 5

In [18]:
for we in we_with_features:
    X, y = prepare_dataset(dataset=we[(we.Gender != 'invariable') & (we.Number != 'invariable') & \
                                      (we.POS.isin(pos))],
                                           feature_col_count=feature_col_count,
                                           feature_name=feature,
                                           normalize=False,
                                           encode=True,
                                           encode_as1='plural',
                                           split=False,
                                           balance=True)
    X_trains = []
    y_trains = []
    
    X_tests = []
    y_tests = []
    
    X_folds = np.array_split(X, n_folds)
    y_folds = np.array_split(y, n_folds)
    
    for i in range(n_folds):
        X_trains.append(pd.DataFrame(np.concatenate(X_folds[:i] + X_folds[i+1:])))
        y_trains.append(np.concatenate(y_folds[:i] + y_folds[i+1:]))

        X_folds[i].columns = X_folds[i].columns.map(int)
        X_tests.append(X_folds[i])
        y_tests.append(y_folds[i])
        
    
    X_na_train.append(X_trains)
    X_na_test.append(X_tests)
    
    y_na_train.append(y_trains)
    y_na_test.append(y_tests)

In [19]:
dataset_sizes = pd.DataFrame(index=labels)
dataset_sizes['1 fold size'] = [len(x[0]) for x in X_na_test]
dataset_sizes

,1 fold size
flau_small_c,1304
flau_base_u,1553
flau_base_c,1304
flau_large_c,1304
cam_base,538
xlm_large,106
xlm_base,106
bert_base_u,389
distilbert_base,257
bert_base_c,257


## Non-independent dims

Calculate non-independendent dimensions for each of _k_ splits.

ANOVA test with p-value < 0.01:

In [20]:
anova_dims = []

In [21]:
for i in range(len(models)):
    anova_dims.append([])
    for j in range(n_folds):
        anova_dims[i].append(get_anova_dims(X_na_train[i][j], y_na_train[i][j]))

Get dimensions where MI > 0:

In [22]:
mi_dims = []

In [23]:
for i in range(len(models)):
    mi_dims.append([])
    for j in range(n_folds):
        mi_dims[i].append(get_mi_dims(X_na_train[i][j], y_na_train[i][j]))

In [24]:
for i in range(len(models)):
    for j in range(n_folds):
        temp = list(map(lambda x: int(x), mi_dims[i][j]))
        mi_dims[i][j] = temp

Get dimensions that are both found by the ANOVA independency test and MI test:

In [25]:
non_ind_dims = []

In [26]:
for i in range(len(models)):
    non_ind_dims.append([])
    for j in range(n_folds):
        non_ind_dims[i].append(list(set(anova_dims[i][j]).intersection(mi_dims[i][j])))

Stats about the number of dimensions for each model for the 1st split:

In [27]:
non_ind_df = pd.DataFrame(index=labels, columns=['ANOVA', 'MI', 'Total non independent'])

In [28]:
non_ind_df['ANOVA'] = [len(x[0]) for x in anova_dims]
non_ind_df['MI'] = [len(x[0]) for x in mi_dims]
non_ind_df['Total non independent'] = [len(x[0]) for x in non_ind_dims]

In [29]:
non_ind_df

,ANOVA,MI,Total non independent
flau_small_c,388,366,307
flau_base_u,339,448,208
flau_base_c,499,542,387
flau_large_c,831,788,689
cam_base,12,362,5
xlm_large,175,593,122
xlm_base,46,417,31
bert_base_u,566,643,480
distilbert_base,277,595,240
bert_base_c,1,431,0


In [30]:
dims = {}

In [31]:
for i in range(len(models)):
    dims[labels[i]] = {}
    dims[labels[i]]['All dims'] = [X_na_train[i][0].columns]*n_folds
    dims[labels[i]]['ANOVA'] = anova_dims[i]
    dims[labels[i]]['MI'] = mi_dims[i]
    dims[labels[i]]['All non ind'] = non_ind_dims[i]

## Important dims

We can test different $\alpha$ values: 1%, 5%, 10%, 25%, 50%, 75%.

In [32]:
alphas = [1, 5, 10, 25, 50, 75]

Train Logistic Regression on train set for each model:

In [33]:
lr_res = []

In [34]:
for i in range(len(models)):
    lr_res.append([])
    for j in range(n_folds):
        lr_res[i].append(lr(X_na_train[i][j], y_na_train[i][j]))

Train Perceptron:

In [35]:
perceptron_res = []

In [36]:
for i in range(len(models)):
    perceptron_res.append([])
    for j in range(n_folds):
        perceptron_res[i].append(perceptron(X_na_train[i][j], y_na_train[i][j]))

Compute correlation to the gender vector:

In [37]:
corr_res = []

In [38]:
for i in range(len(models)):
    corr_res.append([])
    for j in range(n_folds):
        corr_res[i].append(correlation(X_na_train[i][j], y_na_train[i][j]))

In [39]:
for i in range(len(models)):
    for alpha in alphas:
        dims[labels[i]][f'LR{alpha}'] = []
        dims[labels[i]][f'Perc{alpha}'] = []
        dims[labels[i]][f'Corr{alpha}'] = []
        dims[labels[i]][f'All imp dims{alpha}'] = []
        for j in range(n_folds):
            num_imp_dims = len(X_na_test[i][0].columns)*alpha//100
            lr_dims = [x[0] for x in lr_res[i][j][:num_imp_dims]]
            perc_dims = [x[0] for x in perceptron_res[i][j][:num_imp_dims]]
            corr_dims = [x[0] for x in corr_res[i][j][:num_imp_dims]]
            dims[labels[i]][f'LR{alpha}'].append(lr_dims)
            dims[labels[i]][f'Perc{alpha}'].append(perc_dims)
            dims[labels[i]][f'Corr{alpha}'].append(corr_dims)
            dims[labels[i]][f'All imp dims{alpha}'].append(list(set(lr_dims).intersection(perc_dims).intersection(corr_dims)))

## Compute medians

For each of the dimension groups above, compute a median vector: the one that should describe the expected values of the dimensions the best.
We compute median vector associated with feature vector values = 0 and another one for feature vector values = 1.

In [41]:
medians = {}

In [42]:
for i in range(len(models)):
    medians[labels[i]] = {}
    for dim_group in dims[labels[i]].keys():
        medians[labels[i]][dim_group] = []
        n = len(dims[labels[i]][dim_group])
        for j in range(n_folds):
            dim_list = dims[labels[i]][dim_group][j] if n == n_folds else list(dims[labels[i]][dim_group])
            # Median of dimensions where feature vector is equal to 0
            median_0 = X_na_train[i][j][y_na_train[i][j] == 0][dim_list].median()
            # Median of dimensions where feature vector is equal to 1
            median_1 = X_na_train[i][j][y_na_train[i][j] == 1][dim_list].median()

            medians[labels[i]][dim_group].append({
                    '0': median_0,
                    '1': median_1
                })

We can compare the number of dimensions found by each test for each model in the first fold:

In [43]:
dim_lens = {}

for model in dims.keys():
    dim_lens[model] = {}
    for dim_group in dims[model].keys():
        dim_lens[model][dim_group] = len(dims[model][dim_group][0]) if dim_group != 'All dims' else \
                len(dims[model][dim_group])

In [44]:
pd.DataFrame(dim_lens)

,flau_small_c,flau_base_u,flau_base_c,flau_large_c,cam_base,xlm_large,xlm_base,bert_base_u,distilbert_base,bert_base_c
All dims,5,5,5,5,5,5,5,5,5,5
ANOVA,388,339,499,831,12,175,46,566,277,1
MI,366,448,542,788,362,593,417,643,595,431
All non ind,307,208,387,689,5,122,31,480,240,0
LR1,5,7,7,10,7,10,7,7,7,7
Perc1,5,7,7,10,7,10,7,7,7,7
Corr1,5,7,7,10,7,10,7,7,7,7
All imp dims1,2,1,1,2,0,0,0,0,2,0
LR5,25,38,38,51,38,51,38,38,38,38
Perc5,25,38,38,51,38,51,38,38,38,38


## Compute predictions

Using the median vectors computed for each group above, try to predict the value of the feature vector on test.
For this, for any word embedding in test compute distances to median for feature vector values = 0 and for feature vector values = 1. 
If the given word embedding is closer to median of feature vector values = 0, assign 0 as predicted label, 1 otherwise.

In [45]:
y_preds = {}

In [46]:
for i in range(len(models)):
    y_preds[labels[i]] = {}
    for dim_group in dims[labels[i]].keys():
        y_preds[labels[i]][dim_group] = []
        n = len(dims[labels[i]][dim_group])
        for j in range(n_folds):
            dim_list = dims[labels[i]][dim_group][j] if n == n_folds else dims[labels[i]][dim_group]
            
            # If MSE for sample 0 is lower than for sample 1, the label should be 0. So we need to convert False to 0.
            mae0 = X_na_test[i][j][dim_list].apply(lambda x: mean_absolute_error(medians[labels[i]][dim_group][j]['0'], x), axis=1)
            mae1 = X_na_test[i][j][dim_list].apply(lambda x: mean_absolute_error(medians[labels[i]][dim_group][j]['1'], x), axis=1)
            
            y_preds[labels[i]][dim_group].append((mae0 > mae1).apply(int))
    

## Compute accuracies

In [47]:
accs = {}

In [48]:
for i in range(len(models)):
    accs[labels[i]] = {}
    for dim_group in dims[labels[i]].keys():
        accs[labels[i]][dim_group] = []
        for j in range(n_folds):
            y_true = y_na_test[i][j]
            y_pred = y_preds[labels[i]][dim_group][j]
            if any(y_pred):
                acc = accuracy_score(y_true, y_pred)
            else:
                acc = 0
            accs[labels[i]][dim_group].append(acc)

In [49]:
# Compute average accuracy between 5 folds
avg_accs = {}

In [50]:
for i in range(len(models)):
    avg_accs[labels[i]] = {}
    for dim_group in dims[labels[i]].keys():
        avg_accs[labels[i]][dim_group] = np.average(accs[labels[i]][dim_group])

In [51]:
accs_df = pd.DataFrame(avg_accs)
accs_df

,flau_small_c,flau_base_u,flau_base_c,flau_large_c,cam_base,xlm_large,xlm_base,bert_base_u,distilbert_base,bert_base_c
All dims,0.899631,0.617980,0.592540,0.938765,0.514153,0.585157,0.535975,0.561283,0.647206,0.535804
ANOVA,0.901780,0.622617,0.594842,0.939533,0.508188,0.581366,0.556819,0.561283,0.637080,0.534259
MI,0.899786,0.615534,0.601134,0.939072,0.502984,0.588931,0.535993,0.561283,0.650304,0.538138
All non ind,0.899939,0.615017,0.602362,0.938765,0.496653,0.581366,0.554933,0.561797,0.644866,0.431536
LR1,0.756906,0.551262,0.624460,0.818447,0.499994,0.598365,0.583378,0.556649,0.623814,0.549058
Perc1,0.748313,0.568906,0.688458,0.874923,0.507821,0.600341,0.564367,0.557683,0.646410,0.572383
Corr1,0.751532,0.574830,0.686309,0.881216,0.509299,0.602228,0.583396,0.556655,0.627739,0.558384
All imp dims1,0.705495,0.222151,0.637351,0.765497,0.099069,0.111321,0.109434,0.434262,0.360311,0.000000
LR5,0.855740,0.597504,0.627225,0.927410,0.517118,0.587026,0.553064,0.559741,0.666677,0.567735
Perc5,0.859884,0.609222,0.697052,0.932321,0.518246,0.590800,0.547314,0.559739,0.634776,0.543583


We can show what is the accuracy gain for each dimension test comparison to using all dimensions:

In [52]:
gains_df = (accs_df - accs_df.loc['All dims'])
gains_df

,flau_small_c,flau_base_u,flau_base_c,flau_large_c,cam_base,xlm_large,xlm_base,bert_base_u,distilbert_base,bert_base_c
All dims,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ANOVA,0.002149,0.004636,0.002302,0.000767,-0.005965,-0.003792,0.020845,0.000000,-0.010126,-0.001544
MI,0.000155,-0.002447,0.008595,0.000307,-0.011169,0.003774,0.000018,0.000000,0.003098,0.002335
All non ind,0.000309,-0.002964,0.009823,0.000000,-0.017500,-0.003792,0.018958,0.000514,-0.002341,-0.104268
LR1,-0.142725,-0.066718,0.031920,-0.120318,-0.014158,0.013208,0.047403,-0.004634,-0.023392,0.013254
Perc1,-0.151318,-0.049074,0.095918,-0.063842,-0.006332,0.015184,0.028392,-0.003600,-0.000796,0.036579
Corr1,-0.148099,-0.043150,0.093769,-0.057549,-0.004854,0.017071,0.047421,-0.004629,-0.019467,0.022580
All imp dims1,-0.194136,-0.395830,0.044812,-0.173268,-0.415084,-0.473836,-0.426541,-0.127021,-0.286895,-0.535804
LR5,-0.043891,-0.020477,0.034686,-0.011355,0.002966,0.001869,0.017089,-0.001542,0.019470,0.031931
Perc5,-0.039747,-0.008758,0.104513,-0.006444,0.004093,0.005642,0.011339,-0.001544,-0.012430,0.007779


Here are the dimension groups and the accuracy gain this dimension group provides over all dimensions:

In [53]:
for i in range(len(models)):
    print(labels[i], gains_df.idxmax()[i], gains_df.loc[gains_df.idxmax()[i], labels[i]])

flau_small_c LR50 0.04251138241622687
flau_base_u Perc25 0.023311382691299065
flau_base_c Perc5 0.1045125924600614
flau_large_c Perc25 0.013505760656154608
cam_base Perc5 0.004093372930987882
xlm_large All imp dims10 0.03033243486073678
xlm_base Corr1 0.04742138364779869
bert_base_u Corr10 0.004125036440251306
distilbert_base LR10 0.03738752431906622
bert_base_c Perc1 0.036578915369649745


Below, you can find the best achieved accuracy and the dimension group that produced it. This value is `InfEnc`.

In [54]:
for i in range(len(models)):
    print(labels[i], accs_df.idxmax()[i], accs_df.loc[accs_df.idxmax()[i], labels[i]])

flau_small_c LR50 0.9421423661300727
flau_base_u Perc25 0.6412917134113554
flau_base_c Perc5 0.6970521072183586
flau_large_c Perc25 0.952270833235243
cam_base Perc5 0.5182460731172076
xlm_large All imp dims10 0.6154896675651392
xlm_base Corr1 0.5833962264150944
bert_base_u Corr10 0.5654082633238809
distilbert_base LR10 0.6845938715953308
bert_base_c Perc1 0.5723826605058365


In [58]:
with open('../Data/best_results/number_na_dims.pickle', 'wb') as f:
    pickle.dump(dims, f)

In [59]:
with open('../Data/best_results/number_na_accs.pickle', 'wb') as f:
    pickle.dump(accs, f)

In [60]:
with open('../Data/best_results/number_na_medians.pickle', 'wb') as f:
    pickle.dump(medians, f)

# Conclusion

(Similar to number of nouns and number of adjectives)
- FlauBERT small and large show very similar results despite FlauBERT large being 10x larger in terms of paramters
- DistilBERT shows much better results for number than for gender, alongside with XLM